# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [3]:
# load data from database
engine = create_engine('sqlite:///drp.db')
df = pd.read_sql_table('messages', engine)

X = df['message'] # Take the message itself as the input variable, i.e. X
Y = df.drop(columns=['id', 'message', 'original', 'genre'])     # ...and the 36 categories, i.e. all columns except the 4 mentioned here, as output classification results, i.e. Y
#Y = df[df.columns[4:]]

print('Shape of X:',X.shape,', Shape of Y:',Y.shape)
df.head()

Shape of X: (26216,) , Shape of Y: (26216, 36)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
# Further specific imports for tokenizer:
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def tokenize(text):
    
    # Step 1: Tokenize text in words:
    tokens = word_tokenize(text)
    # Step 2: Lemmatize tokenized text:
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() # Step 3: In combination with lemmatization, normalize case & strip white spaces (if any)
        cleaned_tokens.append(clean_tok)
        
    return cleaned_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Specific imports:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
# Import of Estimators:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Specific imports:
from sklearn.model_selection import train_test_split

# Split into train & test set; chosen random_state ensures deterministic results!
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
from sklearn.metrics import classification_report

category_names = Y.columns
print(category_names)

Y_pred = pipeline.predict(X_test)

i = 0
for category in category_names:
    print((i+1),'.',category,':')
    print(classification_report(Y_test.iloc[:,i]
                                , Y_pred[:,i], target_names=category_names))
    i += 1

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')
1 . related :
                        precision    recall  f1-score   support

               related       0.61      0.34      0.44      1563
               request       0.81      0.93      0.87      4944
                 offer       0.70      0.15      0.25        47

           avg / total       0.76      0.79      0.76      6554

2 . request :
                        precision    recall  f1-score   support

               related 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# First, get list of parameters used by pipeline:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7a643466ba60>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [9]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'clf__estimator__n_estimators': [10, 50],    # Default: 10
    'clf__estimator__min_samples_split': [2, 3]  # Default: 2
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.21574610924626184, total=  27.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   33.7s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.21650900213610008, total=  27.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.2360390601159597, total=  27.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


...

[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.21086359475129693, total=  25.4s
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  9.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, score=0.2259688739700946, total= 1.6min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, score=0.23664937442783032, total= 1.7min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, score=0.23237717424473603, total= 1.6min


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 15.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 50], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

AVG score over 3 folds:

RFC-PARAMETERS			n_estimators
min_samples_split	 10	      50	 100
    2			   0.215	0.237	0.240
    3		 	   0.212	0.233	0.238
    
-> Results:
o Increasing the n_estimators parameter increases the score, especially from 10->50
o Increasing the min_samplles_split parameter from 2 to 3 reduces the score a little bit

=> Best score for n_estimators = 100, min_samples_split = 2

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
Y_pred_cv = cv.predict(X_test)

i = 0
for category in category_names:
    print((i+1),'.',category,':')
    print(classification_report(Y_test.iloc[:,i]
                                , Y_pred_cv[:,i], target_names=category_names))
    i += 1

1 . related :
                        precision    recall  f1-score   support

               related       0.68      0.26      0.37      1563
               request       0.80      0.96      0.87      4944
                 offer       0.67      0.17      0.27        47

           avg / total       0.77      0.79      0.75      6554

2 . request :
                        precision    recall  f1-score   support

               related       0.89      0.99      0.94      5443
               request       0.89      0.41      0.56      1111

           avg / total       0.89      0.89      0.87      6554

3 . offer :
                        precision    recall  f1-score   support

               related       0.99      1.00      1.00      6521
               request       0.00      0.00      0.00        33

           avg / total       0.99      0.99      0.99      6554

4 . aid_related :
                        precision    recall  f1-score   support

               related       0.76   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


In [18]:
cv.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.22276, std: 0.00939, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 10},
 mean: 0.23472, std: 0.00414, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50},
 mean: 0.20980, std: 0.00140, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 10},
 mean: 0.23167, std: 0.00439, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 50}]

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
# Try to further improve by switching ML algorithm from RandomForest to AdaBoost:
from sklearn.ensemble import AdaBoostClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))  # NEW: Use AdaBoost Classifier
    ])

pipeline.fit(X_train, Y_train)
Y_pred_AdaBoost = pipeline.predict(X_test)

i = 0
for category in category_names:
    print((i+1),'.',category,':')
    print(classification_report(Y_test.iloc[:,i]
                                , Y_pred_AdaBoost[:,i], target_names=category_names))
    i += 1

1 . related :
                        precision    recall  f1-score   support

               related       0.57      0.24      0.34      1563
               request       0.79      0.94      0.86      4944
                 offer       0.16      0.06      0.09        47

           avg / total       0.74      0.77      0.73      6554

2 . request :
                        precision    recall  f1-score   support

               related       0.91      0.97      0.94      5443
               request       0.77      0.55      0.64      1111

           avg / total       0.89      0.90      0.89      6554

3 . offer :
                        precision    recall  f1-score   support

               related       0.99      1.00      1.00      6521
               request       0.00      0.00      0.00        33

           avg / total       0.99      0.99      0.99      6554

4 . aid_related :
                        precision    recall  f1-score   support

               related       0.75   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


In [24]:
# CV-Optimize AdaBoost:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7a643466ba60>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [25]:
parameters = {
    'clf__estimator__learning_rate': [0.1, 0.5, 1.0],    # Default: 1.0
    'clf__estimator__n_estimators':  [10, 50, 100]         # Default: 50
}

cv2 = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, score=0.17927982911199267, total=  19.4s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.4s remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, score=0.1821788220933781, total=  19.6s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   49.0s remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, score=0.19179127250534025, total=  19.2s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 


...

[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.2162038449801648, total= 2.2min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 12.2min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.1942325297528227, total=  19.3s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.2027769301190113, total=  19.2s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.20323466585291425, total=  19.2s
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, score=0.22459566676838572, total= 1.2min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, score=0.2354287458040891, total= 1.2min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_esti

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 36.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.1, 0.5, 1.0], 'clf__estimator__n_estimators': [10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [27]:
cv2.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.18442, std: 0.00535, params: {'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 10},
 mean: 0.20720, std: 0.00382, params: {'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 50},
 mean: 0.21183, std: 0.00457, params: {'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 100},
 mean: 0.20008, std: 0.00414, params: {'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 10},
 mean: 0.22983, std: 0.00443, params: {'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 50},
 mean: 0.24321, std: 0.00551, params: {'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 100},
 mean: 0.20735, std: 0.00809, params: {'clf__estimator__learning_rate': 1.0, 'clf__estimator__n_estimators': 10},
 mean: 0.23167, std: 0.00418, params: {'clf__estimator__learning_rate': 1.0, 'clf__estimator__n_estimators': 50},
 mean: 0.23395, std: 0.00716, params: {'clf__estimator__learning_rate': 1.0, 'clf__est

In [28]:
Y_pred_cv2 = cv2.predict(X_test)

i = 0
for category in category_names:
    print((i+1),'.',category,':')
    print(classification_report(Y_test.iloc[:,i]
                                , Y_pred_cv2[:,i], target_names=category_names))
    i += 1

1 . related :
                        precision    recall  f1-score   support

               related       0.65      0.20      0.31      1563
               request       0.79      0.97      0.87      4944
                 offer       0.30      0.06      0.11        47

           avg / total       0.75      0.78      0.73      6554

2 . request :
                        precision    recall  f1-score   support

               related       0.91      0.98      0.94      5443
               request       0.82      0.51      0.63      1111

           avg / total       0.89      0.90      0.89      6554

3 . offer :
                        precision    recall  f1-score   support

               related       0.99      1.00      1.00      6521
               request       0.00      0.00      0.00        33

           avg / total       0.99      0.99      0.99      6554

4 . aid_related :
                        precision    recall  f1-score   support

               related       0.76   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 9. Export your model as a pickle file

Since even non-CV-optimized AdaBoost seemed to perform better than CV-optimized RandomForest, the CV-optimized AdaBoost pipeline was selected as model to be exported to pickle!

In [30]:
import pickle

#building new model
opt_model = cv2.best_estimator_
print (cv2.best_estimator_)

with open('drp-classifier.pkl', 'wb') as file:
    pickle.dump(opt_model, file)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ator=None,
          learning_rate=0.5, n_estimators=100, random_state=None),
           n_jobs=1))])


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.